In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
#data collection and representation, use linear regression

In [ ]:
weight=0.7
bias=0.3
#create
X=torch.arange(0,1,0.02).unsqueeze(dim=1)
y=weight*X + bias
X[:10],y[:10]


In [ ]:
#train, validation, test
train_split=int(0.8*len(X))
X_train,y_train=X[:train_split],y[:train_split]
X_test,y_test=X[train_split:],y[train_split:]


In [ ]:
def plot_pred(train_data=X_train, train_labels=y_train,test_labels=y_test, predictions=None):
  plt.figure(figsize=(10,7))
  plt.scatter(X_train,y_train,c="b",s=4,label="training data")
  #predictions
  if predictions is not None:
    plt.scatter(X_test,predictions,c="r",s=4,label="predictions")
    plt.scatter(X_test,y_test,c="g",s=4,label="test data")
  else:
    plt.scatter(X_test,y_test,c="g",s=4,label="test data")
  plt.legend(prop={"size":14})

In [ ]:
plot_pred(X_train, y_train, y_test)

In [ ]:
class linear_regression(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights=nn.Parameter(torch.randn(1,
                                           requires_grad=True,
                                           dtype=torch.float))
    self.bias=nn.Parameter(torch.randn(1,
                                       requires_grad=True,
                                       dtype=torch.float))

    #forward method
  def forward(self,x:torch.Tensor)->torch.Tensor:#-<- x is the input data
    return self.weights*x + self.bias #linear regression formula


###### explanation: model starts with random vlaues(weights,bias), look at trianing data and adjustng the random values to better represent (or get close to) the ideal values, using:
- gradient descent(requires_grad=true)
- back propagation

##pytorch models building essentials
* torch.nn- contains all of the building for computational graphs(neural network)
* torch.nn.Parameter- what parameters should out model try and learn, often pytorch layer from torch.nn will set these
* torch.nn.Module- the base class for all neural network modules, you should overwrite forward()
* torch.optim- this where the optimizer in pytorch live, they will help with gradient descent
* def forward()- all nn.modules requires you to overwrite forward(), this method defines what happens in forward computation

### for data ready most useful are:
* torchvision.transforms
* torch.utils.data.Dataset
* torch.utils.data.DataLoader

### for build/pick a pretained model:
* torch.nn
* torch.nn.Modules
* torchvision.models
* torch.optim

### evaluate mode:
* torchmetrics

### improve through experimentations:
* torch.utils.tensorboard

In [ ]:
# check content of pytorch model
torch.manual_seed(42)#to get same parameters
#create instance
model_0=linear_regression()
list(model_0.parameters())

In [ ]:
model_0.state_dict()

In [ ]:
# make predictions using torch.no_grad()
with torch.no_grad():
  y_pred=model_0(X_test)

y_pred

In [ ]:
# this code is equivalent to the above code the difference
# is that the above code doesnt track gradient,
# less memory as it is not needed in prediction, only in training:
# y_pred=model_0(X_test) # This line is not needed and causes the error
y_pred = model_0(X_test).detach() # Detach the tensor to remove gradient tracking
y_pred

In [ ]:
plot_pred(predictions=y_pred)

In [ ]:
# make predictions using torch.inference_mode()
with torch.no_grad():
  y_pred=model_0(X_test)

y_pred


When to use torch.inference_mode()

Use this after training is completely done — when you’re deploying or running the model for pure inference (predictions only).

When to use torch.no_grad()

Use this when you’re still in a training workflow, but want to momentarily stop gradient tracking

## training model

### difference between cost function and loss function:
* Loss function: measures how wrong the model’s prediction is for one sample (or one batch)

* Cost function: the average (or total) of all the losses across the entire dataset.
It represents the overall error of the model

#### optimizer: takes into account the loss of a model and adjusts the models parameter

In [ ]:
#loss function
loss=nn.L1Loss()
#optimizer (stochastic gradient descent)
optimizer=torch.optim.SGD(params=model_0.parameters(),
                          lr=0.01) #leaning rate

### training and testing loop
1. loop through the data
2. forward pass (data move through the forward function)-forward propagation
3. calculate loss(compare forward pass predictions to actual labels)
4. optimizer zero grad
5. loss backward- backpropagation to calculate gradient of each parameters of our model with respect to the loss
6. optimizer step(gradient descent)

In [ ]:
torch.manual_seed(42)
#build training loop
epochs=100 #one loop through data, hyper parameter because we have set it

#track values
epoch_count=[]
loss_values=[]
test_loss_values=[]
#loop through data


for epoch in range(epochs):
  #set to training mode
  model_0.train()
  #forward pass
  y_pred=model_0(X_train)
  #calculate loss
  loss_train=loss(y_pred,y_train)
  #print(loss_train)
  #optimizer
  optimizer.zero_grad()#reset optimizer to start fresh in each loop
  #loss backward
  loss_train.backward()
  #optimizer step
  optimizer.step()

  #testing
  model_0.eval()
  with torch.inference_mode():
    #forward pass
    test_pred=model_0(X_test)
    #calculate loss
    test_loss=loss(test_pred,y_test)
    #print(test_loss)
    if epoch%10==0:
      epoch_count.append(epoch)
      loss_values.append(loss_train)
      test_loss_values.append(test_loss)
      print(f"epoch: {epoch} | loss:{loss} | testloss: {test_loss}")



In [ ]:
#plot loss curve
plt.plot(epoch_count,torch.tensor(loss_values).numpy(),label="train loss")
plt.plot(epoch_count, torch.tensor(test_loss_values).numpy(), label="test loss" )
plt.title("loss curve")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()

* eval mode-> turns of gradient tracking
* training mode-> turns on gradient tracking

In [ ]:
with torch.inference_mode():
  y_preds_new=model_0(X_test)

In [ ]:
plot_pred(predictions=y_preds_new)

In [ ]:
model_0.state_dict()

# save model
1. torch.save()
2. torch.load- load a saved object
3. torch.nn.Module.load_state_dict()-> allows to load a models daved in state dictionary

In [ ]:
#save model
from pathlib import Path
#create directory
model_path=Path("models")
model_path.mkdir(parents=True,exist_ok=True)
#create model save path
model_name="01_pytorch_workflow_linearR.pth"
model_save_path=model_path/model_name
model_save_path
#Save state dict
torch.save(obj=model_0.state_dict(),
           f=model_save_path)

In [ ]:
#load in model state dict:
loaded_model=linear_regression()
loaded_model.load_state_dict(torch.load(f=model_save_path))
loaded_model.state_dict()